In [1]:
# from .import constant as c
# from .import radiation as rd
# from .import convection as cv

import constant as c
import radiation as rd
import core as co
import pandas as pd

# Example

In [3]:
import constant as c
import radiation as rd
import core as co
import pandas as pd

### 1. Data Loading and Preprocessing

## 1.1 Load the data
data = pd.read_csv('2407_weather_data.csv', index_col=0, encoding='cp949').dropna(subset='지점명', axis=0).fillna(0)

## 1.2 Extract the data
temp = data['기온(°C)']
windspeed = data['풍속(m/s)']
GHI_MJm2 = data['일사(MJ/m2)']

### 2. Weather Data Post-processing

## 2.1 Convert GHI to DNI and DHI
DNI_H, DHI = rd.GHI_MJm2_to_DNI_H_and_DHI(GHI_MJm2, station='광주(Gwangju)', year=2024, month=7, day=10, local_hour=0)

## 2.2 Temperature unit conversion
temp = co.C2K(temp) # °C to K


### 3. Simulation Parameters

## 3.1 Simulation time
sim_time = co.SimulationTimeParameters(PST = 24, MST = 24, dt = 60, start_time = pd.Timestamp('2024-07-10 00:00:00'))


### 4. Data Interpolation

## 4.1 Interpolate hourly data
temp_interp = co.interpolate_hourly_data(temp, sim_time.dt) # K
windspeed_interp = co.interpolate_hourly_data(windspeed, sim_time.dt) # m/s
DNI_H_interp = co.interpolate_hourly_data(DNI_H, sim_time.dt) # W/m2
DHI_interp = co.interpolate_hourly_data(DHI, sim_time.dt) # W/m2

## 4.2 Trim data to simulation time
temp_interp_cut = temp_interp[:sim_time.tN]  # K
windspeed_interp_cut = windspeed_interp[:sim_time.tN] # m/s
DNI_H_interp_cut = DNI_H_interp[:sim_time.tN] # W/m2
DHI_interp_cut = DHI_interp[:sim_time.tN] # W/m2

### 5. Weather Data (Gwangju)
weather = co.WeatherData(location='광주(Gwangju)',
                      temp=temp_interp_cut, 
                      Vz=windspeed_interp_cut, 
                      DNI_H=DNI_H_interp_cut, 
                      DHI=DHI_interp_cut)


### 6. Building Structure Definition

## 6.1 Wall and Roof Layers
concrete = co.Layer(L=0.2, dx=0.01, k=1.4, c=840, rho=2400)
insulation = co.Layer(L=0.1, dx=0.01, k=0.03, c=840, rho=30)

## 6.2 Walls
# Calculate volume based on the structure
wall_height = 3  # Assume wall height is 3 meters
wall_width = 5  # Assume wall width is 5 meters
wall_area = wall_height * wall_width
roof_area = wall_width * wall_width
room_volume = roof_area * wall_height

wall_south = co.Construction(name='Wall_South', layers=[concrete, insulation], roughness='smooth', Tinit=co.C2K(20), area=wall_area, azimuth=0, tilt=90)
wall_east = co.Construction(name='Wall_East', layers=[concrete, insulation], roughness='smooth', Tinit=co.C2K(20), area=wall_area, azimuth=90, tilt=90)
wall_north = co.Construction(name='Wall_North', layers=[concrete, insulation], roughness='smooth', Tinit=co.C2K(20), area=wall_area, azimuth=180, tilt=90)
wall_west = co.Construction(name='Wall_West', layers=[concrete, insulation], roughness='smooth', Tinit=co.C2K(20), area=wall_area, azimuth=270, tilt=90)

## 6.3 Roof
roof = co.Construction(name='Roof', layers=[concrete, insulation], roughness='smooth', Tinit=co.C2K(20), area=roof_area, azimuth=0, tilt=0)

## 6.4 Structure 
structure = [wall_south, wall_east, wall_north, wall_west, roof]

### 7. Indoor Air Properties

indoor_air = co.IndoorAir(ACH=0.5, h_ci=4, volume=room_volume, temperature=co.C2K(20))

### 8. Simulation
co.simulate_one_node_building_exergy(structure=structure,
                                  simulation_time_parameters = sim_time, 
                                  weather = weather, 
                                  indoor_air = indoor_air,
                                  filepath='.')

Simulation progress: 100%|██████████| 2880/2880 [00:00<00:00, 5246.40it/s]
